<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

<span style="font-size:1.1em;">Hangi veriseti kullanılarak işlem yapılacaksa yukardaki pathlerden biri seçilir ve parametre olarak verilir.</span>

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITH_STEMMER, sep = ';', header = 0)

In [4]:
df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,0,19991991,ekşi itiraf dö dolaş gel kendi çat problem ger...,ENTJ,analysts,E,N,T,J
1,1,19991991,selda bağca ses dinleyebilmek büyük mutluluk s...,ENTJ,analysts,E,N,T,J
2,2,19991991,eski sevgili mutlu olma isteye in hiçbir ayak ...,ENTJ,analysts,E,N,T,J
3,3,19991991,veda eder a not bırakmak farklı acı mutluluk...,ENTJ,analysts,E,N,T,J
4,4,19991991,ingil aksa ara bayıldık konuşa insa sevimli ge...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...,...
524835,643907,zaimoglu,zla ibrahimoviç türki katil,ESFJ,sentinels,E,S,F,J
524836,643908,zaimoglu,tarih büyük yalancı şik operasyo aziz yıldır a...,ESFJ,sentinels,E,S,F,J
524837,643909,zaimoglu,akp chp koalisyo hayali koalisyo,ESFJ,sentinels,E,S,F,J
524838,643910,zaimoglu,trabzonlu insa hamsi diye fenerbahçe taraftar ...,ESFJ,sentinels,E,S,F,J


Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [5]:
min_entry = df.groupby('typeClass', as_index = False).count().min().entry

min_entry


54913

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [6]:
analysts_df = df[df['typeClass'] == 'analysts']
analysts_df = analysts_df.iloc[0 : min_entry]

analysts_df.shape[0]


54913

In [7]:
explorers_df = df[df['typeClass'] == 'explorers']
explorers_df = explorers_df.iloc[0 : min_entry]

explorers_df.shape[0]

54913

In [8]:
sentinels_df = df[df['typeClass'] == 'sentinels']
sentinels_df = sentinels_df.iloc[0 : min_entry]

sentinels_df.shape[0]

54913

In [9]:
diplomats_df = df[df['typeClass'] == 'diplomats']

diplomats_df = diplomats_df.iloc[0 : min_entry]

diplomats_df.shape[0]

54913

Yeni bir dataframe oluşturulur.

In [10]:
equal_entries_df = pd.concat([diplomats_df, sentinels_df, explorers_df, analysts_df]).reset_index(drop=True)
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,240593,1903 24,gtü den evr gün ne soruşturma ıee gtü den açık...,INFJ,diplomats,I,N,F,J
1,240594,1903 24,ekşi sözlük speakingi club istanbul hazır,INFJ,diplomats,I,N,F,J
2,240595,1903 24,personalities com sene sür yapmış infj advo...,INFJ,diplomats,I,N,F,J
3,240596,1903 24,müzik dinlemek hara uyuşturuç aşık dertli eser...,INFJ,diplomats,I,N,F,J
4,240599,1903 24,zengi insa kurabilecek cümle al kar sık arar,INFJ,diplomats,I,N,F,J
...,...,...,...,...,...,...,...,...,...
219647,69519,gulcehatun,ceset gör sözlük yazar okuduk görmedi düşündür...,INTJ,analysts,I,N,T,J
219648,69520,gulcehatun,nişa dük tektaş bala istemeye istemeye mümk is...,INTJ,analysts,I,N,T,J
219649,69522,gulcehatun,yürüye merdive duran uyarıl kampanyas istanbul...,INTJ,analysts,I,N,T,J
219650,69523,gulcehatun,zaman baş atla in mesel ağladık gözyaş ke silm...,INTJ,analysts,I,N,T,J


Oluşturulan dataframe shuffle edilir.

In [0]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)

In [12]:
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,476539,ben seni severim sevmesine de,yazar küçük etkil tasarruf yöntem yıl dışarı y...,ISFP,explorers,I,S,F,P
1,478070,bisey demiyorum pes diyorum,ekşi itiraf kendi bahsetmek istiyor bahsedecek...,ESFP,explorers,E,S,F,P
2,481065,damatferid,teknisye tekniker mühendis aras fark gün,ESTP,explorers,E,S,T,P
3,281742,corwin,ses veriyor koorkmaa ilköğret müzik hoca se ok...,ENFP,diplomats,E,N,F,P
4,556204,dumi,fatih ter in fetö belgesel konuşma iftira atan...,ESTJ,sentinels,E,S,T,J
...,...,...,...,...,...,...,...,...,...
219647,39688,canabag,kadı erkek aradık önemli görecel dir başlık be...,INTP,analysts,I,N,T,P
219648,27475,budaboyle,comfortably numb gün can çektik parça ada hiss...,ENTP,analysts,E,N,T,P
219649,587689,lmrvnx,zerr özer in hak değer görememes www youtube...,ISTJ,sentinels,I,S,T,J
219650,549001,civiliduvarkrosesi,in olma iş yaramamas ol ok yaba atar ol yay el...,ESTJ,sentinels,E,S,T,J


<span style="font-size:1.1em;">TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.</span>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='utf-8', ngram_range=(1, 2), max_features=100, analyzer = 'word', token_pattern=r'\w{1,}')

**Feature Extraction** 

Burada tf-idf vektörlerinin çıkartılmasında kullanılacak olan vocabulary oluşturulur.

In [0]:
import numpy as np

equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

In [15]:
tfidf_vectorizer.fit(equal_entries_df['entry'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=100,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['typeClass'], random_state = 42, test_size = 0.20, shuffle = True)

y_actual = y_test.values

In [18]:
X_train.shape[0]

175721

In [19]:
X_test.shape[0]

43931

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [0]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [21]:
X_train_tfidf

<175721x100 sparse matrix of type '<class 'numpy.float64'>'
	with 699906 stored elements in Compressed Sparse Row format>

In [22]:
X_test_tfidf

<43931x100 sparse matrix of type '<class 'numpy.float64'>'
	with 174281 stored elements in Compressed Sparse Row format>

SVM kullanılarak tahmin edilir.

In [23]:
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_predicted = SVM.predict(X_test_tfidf)

In [25]:
y_predicted

array(['explorers', 'diplomats', 'explorers', ..., 'sentinels',
       'explorers', 'explorers'], dtype=object)

<span style="font-size:1.1em">Yapılacak tahminlerle ilgili istatistiksel verileri tutmak için</span> ```predictions_result```<span style="font-size:1.1em"> adında bir değişken oluşturulur.</span>

<span style="font-size:1.1em">Bu değişkenin yapısı aşağıdaki gibidir.</span>

```json
{
    "predicted": {
        "analysts":  { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "diplomats": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "explorers": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "sentinels": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
    }
}
```

* <span style="font-size:1.1em;">Yapılan tahminlerle ilgili verilere ulaşabilmek için</span>

    ```predictions_results['predicted']```


* <span style="font-size:1.1em;">Yapılan tahminin analyst ise:</span>

    ```predictions_results['predicted']['analysts']``` 


* <span style="font-size:1.1em;">Yapılan analyst tahmininin gerçek değerlerine erişmek için:</span>     

    ```predictions_results['predicted']['analysts']['actual']```  


* <span style="font-size:1.1em;">Test verisi, model tarafından analysts olarak tahmin edilmiştir ve bu verinin gerçek değeri de analysts'tir.</span>

    ```predictions_results['predicted']['analysts']['actual']['analysts']``` 

In [0]:
prediction_results = {'predicted': {}}  ## prediction_result['analysts'] means prediction is 'analysts'

prediction_results['predicted']['analysts']  = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['diplomats'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['explorers'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['sentinels'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}

## prediction_result['analysts']['diplomats'] means prediction is analysts but actual value is diplomats

```prediction_results```<span style="font-size:1.1em"> içerisinde tutulan sayaçların değerleri arttırılır.</span>

In [0]:
for i in range(len(y_predicted)):
  predicted_value = y_predicted[i]
  actual_value = y_actual[i]
  prediction_results['predicted'][predicted_value]['actual'][actual_value] += 1

<span style="font-size:1.1em">JSON formatına çevrilir </span>```dict``` <span style="font-size:1.1em">tipi. Bu sayede daha okunaklı bir şekilde print edilmiş olur. </span>

In [28]:
import json

print(json.dumps(prediction_results, indent = 2))

{
  "predicted": {
    "analysts": {
      "actual": {
        "analysts": 2325,
        "diplomats": 1905,
        "explorers": 1886,
        "sentinels": 1973
      }
    },
    "diplomats": {
      "actual": {
        "analysts": 971,
        "diplomats": 1604,
        "explorers": 940,
        "sentinels": 928
      }
    },
    "explorers": {
      "actual": {
        "analysts": 5378,
        "diplomats": 5026,
        "explorers": 5732,
        "sentinels": 5078
      }
    },
    "sentinels": {
      "actual": {
        "analysts": 2457,
        "diplomats": 2430,
        "explorers": 2324,
        "sentinels": 2974
      }
    }
  }
}


<span style="font-size:1.1em;">İlgili field extract edilir</span> ```dict``` <span style="font-size:1.1em;">yapısından.</span>

In [0]:
results = prediction_results['predicted']

<span style="font-size:1.1em;">Başarı oranı hesaplanır</span>

In [30]:
accuracy = (results['analysts']['actual']['analysts'] + results['diplomats']['actual']['diplomats'] + results['explorers']['actual']['explorers'] + results['sentinels']['actual']['sentinels']) / len(y_predicted)
accuracy

0.28761011586351326

<span style="font-size:1.1em;">**E/I** boyutu tahmin edilir</span>

Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [31]:
min_entry = df.groupby('I/E', as_index = False).count().min().entry

min_entry


258860

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [33]:
I_df = df[df['I/E'] == 'I']
I_df = I_df.iloc[0 : min_entry]

I_df.shape[0]


258860

In [34]:
E_df = df[df['I/E'] == 'E']

E_df = E_df.iloc[0 : min_entry]

E_df.shape[0]

258860

Yeni bir dataframe oluşturulur.

In [35]:
equal_entries_df = pd.concat([I_df, E_df]).reset_index(drop=True)
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,1109,Punisher 69,sözlük yazar kız verecek isim arap ismi olmayacak,INTJ,analysts,I,N,T,J
1,1110,Punisher 69,suriyeli göndermeyecek vatandaşlık verecek ülk...,INTJ,analysts,I,N,T,J
2,1111,Punisher 69,klaus animasyo noel temal film seven önerilir ...,INTJ,analysts,I,N,T,J
3,1112,Punisher 69,diamondu tema zaman keşfettik youtube din mito...,INTJ,analysts,I,N,T,J
4,1113,Punisher 69,fatih tezca ın atatürk tweet i atatürk sik yet...,INTJ,analysts,I,N,T,J
...,...,...,...,...,...,...,...,...,...
517715,615066,serendiptytr,şak mak herkes ipho x operatör taksit al...,ESTJ,sentinels,E,S,T,J
517716,615067,serendiptytr,üniversi mezun işsiz kalma sebep beğenme dur k...,ESTJ,sentinels,E,S,T,J
517717,615068,serendiptytr,ramaza ala yemek içmek yasaklanmal insa hayat ...,ESTJ,sentinels,E,S,T,J
517718,615069,serendiptytr,fetö yü kel anlat torba göz meymenetsiz nursuz,ESTJ,sentinels,E,S,T,J


In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['I/E'], random_state = 42, shuffle = True)  ## Geri kalanlar S, T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [38]:
y_predicted

array(['E', 'I', 'E', ..., 'I', 'I', 'I'], dtype=object)

In [39]:
predicted = {}
predicted['I'] = {'actual': {'I': 0, 'E': 0}}
predicted['E'] = {'actual': {'I': 0, 'E': 0}}
predicted

{'E': {'actual': {'E': 0, 'I': 0}}, 'I': {'actual': {'E': 0, 'I': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1


In [41]:
predicted

{'E': {'actual': {'E': 20331, 'I': 15686}},
 'I': {'actual': {'E': 44497, 'I': 48916}}}

In [0]:
accuracy = (predicted['E']['actual']['E'] + predicted['I']['actual']['I']) / len(y_predicted)

In [43]:
accuracy

0.5350150660588735

<span style="font-size:1.1em">**S/N** boyutu tahmin edilir.</span>

Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [44]:
min_entry = df.groupby('S/N', as_index = False).count().min().entry

min_entry


147704

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [46]:
S_df = df[df['S/N'] == 'S']
S_df = S_df.iloc[0 : min_entry]

S_df.shape[0]


147704

In [47]:
N_df = df[df['S/N'] == 'N']

N_df = N_df.iloc[0 : min_entry]

N_df.shape[0]

147704

Yeni bir dataframe oluşturulur.

In [48]:
equal_entries_df = pd.concat([S_df, N_df]).reset_index(drop=True)
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,462663,95lipol1sx,sel karaç çevik ko sosyal medya kullanır payla...,ESFP,explorers,E,S,F,P
1,462664,95lipol1sx,milli anne anlatmıyor oğl,ESFP,explorers,E,S,F,P
2,462665,95lipol1sx,okuna kitap goethe faust,ESFP,explorers,E,S,F,P
3,462666,95lipol1sx,bor seçk üsküdar mühipza celal sahne naşit bey...,ESFP,explorers,E,S,F,P
4,462667,95lipol1sx,ikis kanser anne kız evsiz kalmas,ESFP,explorers,E,S,F,P
...,...,...,...,...,...,...,...,...,...
295403,187168,sanal gezgin,decathlo dan alamaya sporsuz in kas yapma ihti...,INTP,analysts,I,N,T,P
295404,187169,sanal gezgin,tecavüz engelledik iş genç olay yer ayr verilm...,INTP,analysts,I,N,T,P
295405,187170,sanal gezgin,fenerbahçe tarih gereksiz transfer selçuk mehm...,INTP,analysts,I,N,T,P
295406,187171,sanal gezgin,ekono yönet g kurtarma dönük çözüm peş idar or...,INTP,analysts,I,N,T,P


In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['S/N'], random_state = 42, shuffle = True)  ## Geri kalan boyutlar: T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['N'] = {'actual': {'N': 0, 'S': 0}}
predicted['S'] = {'actual': {'N': 0, 'S': 0}}

predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [53]:
predicted

{'E': {'actual': {'E': 20331, 'I': 15686}},
 'I': {'actual': {'E': 44497, 'I': 48916}},
 'N': {'actual': {'N': 13275, 'S': 11089}},
 'S': {'actual': {'N': 23709, 'S': 25779}}}

In [0]:
accuracy = (predicted['N']['actual']['N'] + predicted['S']['actual']['S']) / len(y_predicted)

In [55]:
accuracy

0.5288143855278123

<span style="font-size:1.1em">**T/F** boyutu tahmin edilir.</span>

Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [56]:
min_entry = df.groupby('T/F', as_index = False).count().min().entry

min_entry


249821

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [57]:
T_df = df[df['T/F'] == 'T']
T_df = T_df.iloc[0 : min_entry]

T_df.shape[0]


249821

In [58]:
F_df = df[df['T/F'] == 'F']

F_df = F_df.iloc[0 : min_entry]

F_df.shape[0]

249821

Yeni bir dataframe oluşturulur.

In [59]:
equal_entries_df = pd.concat([T_df, F_df]).reset_index(drop=True)
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,0,19991991,ekşi itiraf dö dolaş gel kendi çat problem ger...,ENTJ,analysts,E,N,T,J
1,1,19991991,selda bağca ses dinleyebilmek büyük mutluluk s...,ENTJ,analysts,E,N,T,J
2,2,19991991,eski sevgili mutlu olma isteye in hiçbir ayak ...,ENTJ,analysts,E,N,T,J
3,3,19991991,veda eder a not bırakmak farklı acı mutluluk...,ENTJ,analysts,E,N,T,J
4,4,19991991,ingil aksa ara bayıldık konuşa insa sevimli ge...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...,...
499637,643906,zaimoglu,enstrüma çala sözlük yazar sözlük ol se bağl...,ESFJ,sentinels,E,S,F,J
499638,643907,zaimoglu,zla ibrahimoviç türki katil,ESFJ,sentinels,E,S,F,J
499639,643908,zaimoglu,tarih büyük yalancı şik operasyo aziz yıldır a...,ESFJ,sentinels,E,S,F,J
499640,643909,zaimoglu,akp chp koalisyo hayali koalisyo,ESFJ,sentinels,E,S,F,J


In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['T/F'], random_state = 42, shuffle = True)  ##  J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [61]:
predicted['T'] = {'actual': {'T': 0, 'F': 0}}
predicted['F'] = {'actual': {'T': 0, 'F': 0}}

predicted

{'E': {'actual': {'E': 20331, 'I': 15686}},
 'F': {'actual': {'F': 0, 'T': 0}},
 'I': {'actual': {'E': 44497, 'I': 48916}},
 'N': {'actual': {'N': 13275, 'S': 11089}},
 'S': {'actual': {'N': 23709, 'S': 25779}},
 'T': {'actual': {'F': 0, 'T': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [63]:
predicted

{'E': {'actual': {'E': 20331, 'I': 15686}},
 'F': {'actual': {'F': 32058, 'T': 26798}},
 'I': {'actual': {'E': 44497, 'I': 48916}},
 'N': {'actual': {'N': 13275, 'S': 11089}},
 'S': {'actual': {'N': 23709, 'S': 25779}},
 'T': {'actual': {'F': 30479, 'T': 35576}}}

In [0]:
accuracy = (predicted['F']['actual']['F'] + predicted['T']['actual']['T']) / len(y_predicted)

In [65]:
accuracy

0.5414575177526398

<span style="font-size:1.1em">**J/P** boyutu tahmin edilir.</span>

Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [66]:
min_entry = df.groupby('J/P', as_index = False).count().min().entry

min_entry


255875

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [67]:
J_df = df[df['J/P'] == 'J']
J_df = J_df.iloc[0 : min_entry]

J_df.shape[0]


255875

In [68]:
P_df = df[df['J/P'] == 'P']

P_df = P_df.iloc[0 : min_entry]

P_df.shape[0]

255875

Yeni bir dataframe oluşturulur.

In [69]:
equal_entries_df = pd.concat([I_df, E_df]).reset_index(drop=True)
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,1109,Punisher 69,sözlük yazar kız verecek isim arap ismi olmayacak,INTJ,analysts,I,N,T,J
1,1110,Punisher 69,suriyeli göndermeyecek vatandaşlık verecek ülk...,INTJ,analysts,I,N,T,J
2,1111,Punisher 69,klaus animasyo noel temal film seven önerilir ...,INTJ,analysts,I,N,T,J
3,1112,Punisher 69,diamondu tema zaman keşfettik youtube din mito...,INTJ,analysts,I,N,T,J
4,1113,Punisher 69,fatih tezca ın atatürk tweet i atatürk sik yet...,INTJ,analysts,I,N,T,J
...,...,...,...,...,...,...,...,...,...
517715,615066,serendiptytr,şak mak herkes ipho x operatör taksit al...,ESTJ,sentinels,E,S,T,J
517716,615067,serendiptytr,üniversi mezun işsiz kalma sebep beğenme dur k...,ESTJ,sentinels,E,S,T,J
517717,615068,serendiptytr,ramaza ala yemek içmek yasaklanmal insa hayat ...,ESTJ,sentinels,E,S,T,J
517718,615069,serendiptytr,fetö yü kel anlat torba göz meymenetsiz nursuz,ESTJ,sentinels,E,S,T,J


In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['J/P'], random_state = 42, shuffle = True) 

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['J'] = {'actual': {'J': 0, 'P': 0}}
predicted['P'] = {'actual': {'J': 0, 'P': 0}}

predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

In [0]:
accuracy = (predicted['P']['actual']['P'] + predicted['J']['actual']['J']) / len(y_predicted)

In [0]:
accuracy

In [0]:
prediction_results['predicted'].update(predicted) 

prediction_results